# **Chapter 40: Security Testing Techniques**

---

## **40.1 OWASP Testing Guide Methodology**

### **40.1.1 Overview of OWASP Testing Framework**

The OWASP Testing Guide provides a comprehensive framework for testing web application security. It represents the industry standard for structured security assessment, organized into 12 categories covering all aspects of web application security.

**The 12 Testing Categories:**

```
OWASP Testing Framework:

1. Information Gathering          7. Cryptography
2. Configuration and Deployment   8. Error Handling
3. Identity Management            9. Business Logic
4. Authentication                 10. Client-side
5. Authorization                  11. API Testing
6. Session Management             12. Denial of Service
```

### **40.1.2 Information Gathering (OTG-INFO)**

Information gathering is the foundation of security testing. The more you know about the target, the more effective your testing will be.

**Reconnaissance Techniques:**

```python
# Automated Information Gathering Script
import requests
import dns.resolver
import whois
import socket
import ssl
import subprocess
from bs4 import BeautifulSoup
import json

class InformationGathering:
    def __init__(self, target):
        self.target = target
        self.base_url = f"http://{target}" if not target.startswith('http') else target
        self.findings = {}
    
    def dns_enumeration(self):
        """
        DNS reconnaissance and subdomain discovery
        """
        dns_info = {
            'a_records': [],
            'mx_records': [],
            'ns_records': [],
            'txt_records': [],
            'subdomains': []
        }
        
        # Basic DNS records
        try:
            dns_info['a_records'] = [str(rdata) for rdata in dns.resolver.resolve(self.target, 'A')]
            dns_info['mx_records'] = [str(rdata.exchange) for rdata in dns.resolver.resolve(self.target, 'MX')]
            dns_info['ns_records'] = [str(rdata) for rdata in dns.resolver.resolve(self.target, 'NS')]
            dns_info['txt_records'] = [str(rdata) for rdata in dns.resolver.resolve(self.target, 'TXT')]
        except Exception as e:
            print(f"DNS lookup error: {e}")
        
        # Subdomain enumeration using wordlist
        common_subdomains = ['www', 'mail', 'ftp', 'admin', 'api', 'dev', 'test', 'staging', 'blog', 'shop']
        
        for sub in common_subdomains:
            subdomain = f"{sub}.{self.target}"
            try:
                ip = socket.gethostbyname(subdomain)
                dns_info['subdomains'].append({
                    'subdomain': subdomain,
                    'ip': ip
                })
            except socket.gaierror:
                pass
        
        self.findings['dns'] = dns_info
        return dns_info
    
    def web_fingerprinting(self):
        """
        Identify technologies, frameworks, and server information
        """
        try:
            response = requests.get(self.base_url, timeout=10)
            headers = response.headers
            
            fingerprint = {
                'server': headers.get('Server', 'Not disclosed'),
                'powered_by': headers.get('X-Powered-By', 'Not disclosed'),
                'framework': None,
                'cms': None,
                'technologies': []
            }
            
            # Framework detection via headers
            if 'django' in headers.get('Server', '').lower() or 'csrftoken' in str(headers):
                fingerprint['framework'] = 'Django'
                fingerprint['technologies'].append('Python')
            
            if 'express' in headers.get('X-Powered-By', '').lower():
                fingerprint['framework'] = 'Express.js'
                fingerprint['technologies'].append('Node.js')
            
            if 'asp.net' in headers.get('X-Powered-By', '').lower():
                fingerprint['framework'] = 'ASP.NET'
                fingerprint['technologies'].append('.NET')
            
            # CMS Detection via meta tags
            soup = BeautifulSoup(response.text, 'html.parser')
            generator = soup.find('meta', attrs={'name': 'generator'})
            if generator:
                fingerprint['cms'] = generator.get('content')
            
            # Check for common paths
            tech_indicators = {
                'wp-content': 'WordPress',
                'drupal': 'Drupal',
                'joomla': 'Joomla',
                'rails': 'Ruby on Rails',
                'laravel': 'Laravel',
                'react': 'React',
                'vue': 'Vue.js',
                'angular': 'Angular'
            }
            
            for indicator, tech in tech_indicators.items():
                if indicator in response.text.lower():
                    fingerprint['technologies'].append(tech)
            
            # SSL/TLS Information
            try:
                context = ssl.create_default_context()
                with socket.create_connection((self.target, 443), timeout=5) as sock:
                    with context.wrap_socket(sock, server_hostname=self.target) as ssock:
                        cert = ssock.getpeercert()
                        cipher = ssock.cipher()
                        version = ssock.version()
                        
                        fingerprint['ssl'] = {
                            'version': version,
                            'cipher': cipher[0],
                            'cert_issuer': cert.get('issuer'),
                            'cert_subject': cert.get('subject'),
                            'not_after': cert.get('notAfter')
                        }
            except Exception as e:
                fingerprint['ssl'] = {'error': str(e)}
            
            self.findings['fingerprint'] = fingerprint
            return fingerprint
            
        except requests.RequestException as e:
            return {'error': str(e)}
    
    def enumerate_endpoints(self):
        """
        Discover hidden files and directories
        """
        common_paths = [
            '/robots.txt', '/sitemap.xml', '/.git/', '/.env',
            '/admin/', '/login/', '/api/', '/backup/', '/config/',
            '/phpinfo.php', '/.htaccess', '/web.config',
            '/swagger-ui.html', '/api-docs', '/graphql',
            '/actuator/', '/health', '/metrics'
        ]
        
        discovered = []
        
        for path in common_paths:
            url = f"{self.base_url}{path}"
            try:
                response = requests.get(url, timeout=5, allow_redirects=False)
                
                if response.status_code in [200, 401, 403]:
                    discovered.append({
                        'path': path,
                        'status': response.status_code,
                        'content_length': len(response.content),
                        'content_type': response.headers.get('Content-Type')
                    })
                    
                    # Special handling for specific files
                    if path == '/robots.txt':
                        self.parse_robots_txt(response.text)
                        
            except requests.RequestException:
                continue
        
        self.findings['endpoints'] = discovered
        return discovered
    
    def parse_robots_txt(self, content):
        """
        Parse robots.txt for sensitive paths
        """
        disallowed = []
        for line in content.split('\n'):
            if line.lower().startswith('disallow:'):
                path = line.split(':', 1)[1].strip()
                disallowed.append(path)
        
        self.findings['robots_disallowed'] = disallowed
        return disallowed
    
    def whois_lookup(self):
        """
        Domain registration information
        """
        try:
            w = whois.whois(self.target)
            whois_info = {
                'registrar': w.registrar,
                'creation_date': str(w.creation_date),
                'expiration_date': str(w.expiration_date),
                'name_servers': w.name_servers,
                'emails': w.emails,
                'org': w.org
            }
            self.findings['whois'] = whois_info
            return whois_info
        except Exception as e:
            return {'error': str(e)}
    
    def generate_info_report(self):
        """
        Generate comprehensive information gathering report
        """
        report = {
            'target': self.target,
            'scan_date': str(datetime.now()),
            'findings': self.findings,
            'risk_assessment': self.assess_information_exposure()
        }
        
        return report
    
    def assess_information_exposure(self):
        """
        Assess risks from exposed information
        """
        risks = []
        
        if self.findings.get('fingerprint', {}).get('server') != 'Not disclosed':
            risks.append("Server version disclosure - potential for targeted exploits")
        
        if '.git' in str(self.findings.get('endpoints', [])):
            risks.append("CRITICAL: Git repository exposed - source code leakage")
        
        if '.env' in str(self.findings.get('endpoints', [])):
            risks.append("CRITICAL: Environment file exposed - credential leakage")
        
        if 'admin' in str(self.findings.get('endpoints', [])).lower():
            risks.append("Admin interface exposed - brute force target")
        
        return risks

# Usage Example
# scanner = InformationGathering("example.com")
# scanner.dns_enumeration()
# scanner.web_fingerprinting()
# scanner.enumerate_endpoints()
# print(scanner.generate_info_report())
```

### **40.1.3 Configuration and Deployment Management Testing**

Testing for security misconfigurations in the application infrastructure:

```python
# Configuration Security Testing
class ConfigurationTester:
    def __init__(self, target_url):
        self.target = target_url
        self.session = requests.Session()
    
    def test_http_methods(self):
        """
        Test for dangerous HTTP methods (PUT, DELETE, TRACE, etc.)
        """
        methods = ['GET', 'POST', 'PUT', 'DELETE', 'PATCH', 'OPTIONS', 'TRACE', 'HEAD']
        results = {}
        
        for method in methods:
            try:
                response = self.session.request(method, self.target)
                results[method] = {
                    'status': response.status_code,
                    'allowed': response.status_code not in [405, 501],
                    'headers': dict(response.headers)
                }
                
                # Check for TRACE (XST - Cross-Site Tracing)
                if method == 'TRACE' and response.status_code == 200:
                    results[method]['warning'] = 'TRACE enabled - XST vulnerability'
                
            except Exception as e:
                results[method] = {'error': str(e)}
        
        return results
    
    def test_security_headers(self):
        """
        Verify security headers implementation
        """
        required_headers = {
            'Strict-Transport-Security': {
                'required': True,
                'check': lambda v: 'max-age=' in v and int(v.split('max-age=')[1].split(';')[0]) >= 31536000
            },
            'X-Content-Type-Options': {
                'required': True,
                'expected': 'nosniff'
            },
            'X-Frame-Options': {
                'required': True,
                'check': lambda v: v.upper() in ['DENY', 'SAMEORIGIN']
            },
            'Content-Security-Policy': {
                'required': True,
                'check': lambda v: len(v) > 10  # Basic check for presence
            },
            'Referrer-Policy': {
                'required': True
            },
            'Permissions-Policy': {
                'required': False  # Optional but recommended
            }
        }
        
        response = self.session.get(self.target)
        headers = {k.lower(): v for k, v in response.headers.items()}
        
        findings = []
        for header, config in required_headers.items():
            header_lower = header.lower()
            
            if header_lower not in headers:
                if config['required']:
                    findings.append({
                        'header': header,
                        'status': 'MISSING',
                        'severity': 'Medium',
                        'recommendation': f'Add {header} header'
                    })
            else:
                value = headers[header_lower]
                if 'expected' in config and value != config['expected']:
                    findings.append({
                        'header': header,
                        'status': 'MISCONFIGURED',
                        'value': value,
                        'expected': config['expected'],
                        'severity': 'Low'
                    })
                elif 'check' in config and not config['check'](value):
                    findings.append({
                        'header': header,
                        'status': 'WEAK',
                        'value': value,
                        'severity': 'Low'
                    })
        
        return findings
    
    def test_ssl_configuration(self):
        """
        Test SSL/TLS configuration using sslscan or similar
        """
        import ssl
        import socket
        
        hostname = self.target.replace('https://', '').replace('http://', '').split('/')[0]
        
        try:
            context = ssl.create_default_context()
            with socket.create_connection((hostname, 443)) as sock:
                with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                    cert = ssock.getpeercert()
                    cipher = ssock.cipher()
                    version = ssock.version()
                    
                    findings = []
                    
                    # Check SSL/TLS version
                    if version in ['SSLv2', 'SSLv3', 'TLSv1', 'TLSv1.1']:
                        findings.append({
                            'issue': 'Deprecated TLS version',
                            'current': version,
                            'severity': 'High',
                            'recommendation': 'Upgrade to TLS 1.2 or 1.3'
                        })
                    
                    # Check certificate validity
                    from datetime import datetime
                    not_after = cert.get('notAfter')
                    if not_after:
                        expiry = datetime.strptime(not_after, '%b %d %H:%M:%S %Y %Z')
                        if expiry < datetime.now():
                            findings.append({
                                'issue': 'Expired certificate',
                                'expiry': str(expiry),
                                'severity': 'Critical'
                            })
                    
                    return {
                        'version': version,
                        'cipher': cipher,
                        'findings': findings
                    }
        except Exception as e:
            return {'error': str(e)}

# Usage
# tester = ConfigurationTester("https://example.com")
# print(tester.test_security_headers())
```

### **40.1.4 Authentication Testing**

```python
# Authentication Security Testing
class AuthenticationTester:
    def __init__(self, login_url):
        self.login_url = login_url
        self.session = requests.Session()
    
    def test_credential_stuffing_protection(self, usernames, passwords):
        """
        Test for brute force and credential stuffing protection
        """
        results = {
            'rate_limiting': False,
            'account_lockout': False,
            'captcha_present': False,
            'observations': []
        }
        
        # Attempt multiple rapid logins
        for i, (user, pwd) in enumerate(zip(usernames[:5], passwords[:5])):
            start_time = time.time()
            response = self.session.post(
                self.login_url,
                data={'username': user, 'password': pwd}
            )
            duration = time.time() - start_time
            
            # Check for rate limiting (slow response or specific message)
            if duration > 2:  # Arbitrary threshold
                results['rate_limiting'] = True
                results['observations'].append(f"Request {i+1} delayed: {duration:.2f}s")
            
            # Check for account lockout messages
            if 'locked' in response.text.lower() or 'attempts' in response.text.lower():
                results['account_lockout'] = True
            
            # Check for CAPTCHA
            if 'captcha' in response.text.lower():
                results['captcha_present'] = True
        
        return results
    
    def test_password_policy(self):
        """
        Test password complexity requirements
        """
        test_passwords = [
            ('short', 'Too short'),
            ('password', 'Common dictionary word'),
            ('Password1', 'Basic complexity'),
            ('P@ssw0rd', 'Leet speak substitution'),
            ('1234567890', 'Sequential numbers'),
            ('aaaaaaaa', 'Repeated characters')
        ]
        
        registration_url = self.login_url.replace('login', 'register')
        results = []
        
        for pwd, description in test_passwords:
            try:
                response = self.session.post(
                    registration_url,
                    data={
                        'username': f'testuser_{random.randint(1000,9999)}',
                        'password': pwd,
                        'confirm_password': pwd
                    }
                )
                
                results.append({
                    'password': pwd,
                    'description': description,
                    'accepted': 'success' in response.text.lower() or response.status_code == 200,
                    'error_message': response.text if 'error' in response.text.lower() else None
                })
            except Exception as e:
                results.append({'password': pwd, 'error': str(e)})
        
        return results
    
    def test_session_management(self):
        """
        Test session fixation and session hijacking protections
        """
        # Step 1: Get pre-login session cookie
        pre_login = self.session.get(self.login_url)
        pre_login_cookie = self.session.cookies.get('sessionid')
        
        # Step 2: Login
        login_response = self.session.post(
            self.login_url,
            data={'username': 'validuser', 'password': 'validpass'}
        )
        post_login_cookie = self.session.cookies.get('sessionid')
        
        findings = []
        
        # Check if session ID changed after login
        if pre_login_cookie == post_login_cookie:
            findings.append({
                'issue': 'Session Fixation',
                'severity': 'High',
                'description': 'Session ID not regenerated after authentication'
            })
        
        # Check cookie flags
        if 'set-cookie' in login_response.headers:
            cookie_header = login_response.headers['set-cookie'].lower()
            
            if 'httponly' not in cookie_header:
                findings.append({
                    'issue': 'Missing HttpOnly flag',
                    'severity': 'Medium'
                })
            
            if 'secure' not in cookie_header:
                findings.append({
                    'issue': 'Missing Secure flag',
                    'severity': 'Medium'
                })
            
            if 'samesite' not in cookie_header:
                findings.append({
                    'issue': 'Missing SameSite attribute',
                    'severity': 'Low'
                })
        
        return findings
```

---

## **40.2 Penetration Testing Execution Standard (PTES)**

### **40.2.1 PTES Framework Overview**

PTES defines a standard for penetration testing execution that ensures comprehensive and consistent security assessments across 7 phases:

```
PTES Phases:

1. Pre-engagement Interactions
   ├── Scope definition
   ├── Rules of engagement
   ├── Legal considerations
   └── Communication channels

2. Intelligence Gathering
   ├── Open source intelligence (OSINT)
   ├── Human intelligence (HUMINT)
   └── Footprinting

3. Threat Modeling
   ├── Business asset analysis
   ├── Threat agent identification
   └── Attack surface mapping

4. Vulnerability Analysis
   ├── Automated scanning
   ├── Manual verification
   └── Exploit research

5. Exploitation
   ├── Proof of concept development
   ├── Privilege escalation
   └── Persistence establishment

6. Post Exploitation
   ├── Data exfiltration analysis
   ├── Pivoting opportunities
   └── Business impact assessment

7. Reporting
   ├── Technical findings
   ├── Executive summary
   └── Remediation roadmap
```

### **40.2.2 Pre-engagement and Scoping**

```python
# PTES Pre-engagement Documentation Generator
class PTESPreEngagement:
    def __init__(self, client_name, start_date, end_date):
        self.client = client_name
        self.start = start_date
        self.end = end_date
        self.scope = {}
        self.rules = {}
    
    def define_scope(self, in_scope, out_of_scope, limitations):
        """
        Define testing scope with explicit boundaries
        """
        self.scope = {
            'in_scope_assets': in_scope,  # IPs, domains, apps
            'out_of_scope': out_of_scope,
            'testing_limitations': limitations,
            'testing_windows': 'Business hours only / 24x7',
            'excluded_vulnerabilities': [
                'Physical security testing',
                'Social engineering without approval',
                'DoS attacks',
                'Data destruction'
            ]
        }
    
    def generate_roe_document(self):
        """
        Generate Rules of Engagement document
        """
        roe = f"""
        RULES OF ENGAGEMENT
        ==================
        
        Client: {self.client}
        Assessment Period: {self.start} to {self.end}
        
        1. AUTHORIZATION
        This document serves as written authorization for security testing 
        of the defined scope. Any testing outside defined scope is prohibited.
        
        2. SCOPE
        In-Scope Assets:
        {chr(10).join(f"  - {asset}" for asset in self.scope.get('in_scope_assets', []))}
        
        Out-of-Scope:
        {chr(10).join(f"  - {item}" for item in self.scope.get('out_of_scope', []))}
        
        3. TESTING CONSTRAINTS
        - Testing Window: {self.scope.get('testing_windows')}
        - Maximum impact: Non-destructive only
        - Notification requirements: 24 hours for critical findings
        
        4. EMERGENCY CONTACTS
        Primary: [Security Team Lead]
        Secondary: [IT Operations]
        Escalation: [CISO]
        
        5. DELIVERABLES
        - Executive Summary
        - Technical Findings Report
        - Remediation Roadmap
        - Raw scan data (optional)
        
        6. LEGAL CONSIDERATIONS
        - Confidentiality: All data handled per NDA
        - Data retention: 90 days post-delivery, then secure destruction
        - Compliance: Testing aligned with industry regulations (PCI-DSS, HIPAA, etc.)
        
        Signed: _________________ Date: _________________
        Client Representative
        """
        return roe

    def create_testing_matrix(self):
        """
        Create testing methodology matrix per PTES
        """
        matrix = {
            'Information_Gathering': {
                'passive': ['WHOIS', 'DNS enumeration', 'OSINT'],
                'active': ['Port scanning', 'Service enumeration'],
                'tools': ['theHarvester', 'Maltego', 'Nmap'],
                'time_estimate': '8 hours'
            },
            'Vulnerability_Analysis': {
                'automated': ['Nessus', 'OpenVAS', 'Nuclei'],
                'manual': ['Configuration review', 'Version checking'],
                'validation': ['False positive removal', 'Exploitability assessment'],
                'time_estimate': '16 hours'
            },
            'Exploitation': {
                'approach': 'Controlled exploitation with safety measures',
                'tools': ['Metasploit', 'Manual exploits', 'Custom scripts'],
                'constraints': ['No production data modification without approval'],
                'time_estimate': '16 hours'
            },
            'Post_Exploitation': {
                'activities': ['Privilege escalation testing', 'Lateral movement analysis'],
                'data_handling': 'Screenshots and logs only, no sensitive data extraction',
                'time_estimate': '8 hours'
            }
        }
        return matrix
```

### **40.2.3 Threat Modeling**

```python
# Threat Modeling using STRIDE methodology
class ThreatModeling:
    """
    STRIDE Threat Classification:
    - Spoofing
    - Tampering
    - Repudiation
    - Information Disclosure
    - Denial of Service
    - Elevation of Privilege
    """
    
    def __init__(self, application_name, data_flow_diagram=None):
        self.app = application_name
        self.dfd = data_flow_diagram  # Data Flow Diagram
        self.threats = []
    
    def identify_threats(self, component_type, component_name, data_flow):
        """
        Identify threats based on component type
        """
        threats = []
        
        if component_type == 'process':
            threats.extend([
                {'type': 'Spoofing', 'description': f'Impersonation of {component_name}'},
                {'type': 'Tampering', 'description': f'Code modification of {component_name}'},
                {'type': 'Repudiation', 'description': f'Lack of audit logging in {component_name}'},
                {'type': 'Elevation of Privilege', 'description': f'Unauthorized access to {component_name}'}
            ])
        
        elif component_type == 'data_store':
            threats.extend([
                {'type': 'Tampering', 'description': f'Data modification in {component_name}'},
                {'type': 'Information Disclosure', 'description': f'Unauthorized data access in {component_name}'},
                {'type': 'Denial of Service', 'description': f'Resource exhaustion of {component_name}'}
            ])
        
        elif component_type == 'data_flow':
            threats.extend([
                {'type': 'Tampering', 'description': f'Man-in-the-middle attack on {component_name}'},
                {'type': 'Information Disclosure', 'description': f'Eavesdropping on {component_name}'},
                {'type': 'Denial of Service', 'description': f'Traffic flooding of {component_name}'}
            ])
        
        elif component_type == 'external_entity':
            threats.extend([
                {'type': 'Spoofing', 'description': f'Fake {component_name} connection'},
                {'type': 'Repudiation', 'description': f'{component_name} denying actions'}
            ])
        
        for threat in threats:
            threat['component'] = component_name
            threat['mitigation'] = self.suggest_mitigation(threat['type'], component_type)
            self.threats.append(threat)
        
        return threats
    
    def suggest_mitigation(self, threat_type, component_type):
        """
        Suggest mitigations based on threat type
        """
        mitigations = {
            'Spoofing': {
                'process': 'Authentication, Digital signatures',
                'external_entity': 'Certificate pinning, Mutual TLS',
                'default': 'Strong authentication mechanisms'
            },
            'Tampering': {
                'data_flow': 'TLS/SSL, Message authentication codes',
                'data_store': 'Access controls, Integrity checks',
                'process': 'Code signing, Input validation',
                'default': 'Integrity verification'
            },
            'Repudiation': {
                'process': 'Audit logging, Digital signatures, Timestamps',
                'external_entity': 'Secure logging, Non-repudiation protocols',
                'default': 'Comprehensive logging'
            },
            'Information Disclosure': {
                'data_flow': 'Encryption, Access controls',
                'data_store': 'Encryption at rest, Access controls, Data masking',
                'process': 'Minimal privilege, Secure memory handling',
                'default': 'Confidentiality controls'
            },
            'Denial of Service': {
                'data_flow': 'Rate limiting, Traffic filtering',
                'data_store': 'Resource quotas, Redundancy',
                'process': 'Input validation, Resource management',
                'default': 'Availability controls'
            },
            'Elevation of Privilege': {
                'process': 'Authorization checks, Input validation, Sandboxing',
                'default': 'Least privilege, Access controls'
            }
        }
        
        if threat_type in mitigations:
            if component_type in mitigations[threat_type]:
                return mitigations[threat_type][component_type]
            return mitigations[threat_type].get('default', 'Implement security controls')
        
        return 'Review and implement appropriate controls'
    
    def calculate_risk_score(self, threat, likelihood=3, impact=3):
        """
        Calculate risk score: Likelihood (1-5) * Impact (1-5)
        """
        risk_matrix = {
            (1, 1): 'Low', (1, 2): 'Low', (1, 3): 'Low',
            (2, 1): 'Low', (2, 2): 'Medium', (2, 3): 'Medium',
            (3, 1): 'Low', (3, 2): 'Medium', (3, 3): 'High',
            (4, 1): 'Medium', (4, 2): 'High', (4, 3): 'Critical',
            (5, 1): 'Medium', (5, 2): 'High', (5, 3): 'Critical'
        }
        
        score = likelihood * impact
        severity = risk_matrix.get((likelihood, impact), 'Unknown')
        
        return {
            'score': score,
            'severity': severity,
            'likelihood': likelihood,
            'impact': impact
        }
    
    def generate_threat_model_report(self):
        """
        Generate comprehensive threat model document
        """
        report = {
            'application': self.app,
            'methodology': 'STRIDE',
            'total_threats': len(self.threats),
            'threats_by_category': {},
            'threats': self.threats
        }
        
        # Categorize threats
        for threat in self.threats:
            category = threat['type']
            if category not in report['threats_by_category']:
                report['threats_by_category'][category] = 0
            report['threats_by_category'][category] += 1
        
        return report
```

---

## **40.3 API Security Testing**

### **40.3.1 REST API Security Testing**

```python
# Comprehensive API Security Testing
class APISecurityTester:
    def __init__(self, base_url, api_key=None):
        self.base_url = base_url
        self.session = requests.Session()
        if api_key:
            self.session.headers.update({'Authorization': f'Bearer {api_key}'})
    
    def test_authentication_bypass(self, endpoints):
        """
        Test for authentication bypass vulnerabilities
        """
        results = []
        
        for endpoint in endpoints:
            # Test 1: No authentication
            response = requests.get(f"{self.base_url}{endpoint}")
            if response.status_code != 401:
                results.append({
                    'endpoint': endpoint,
                    'issue': 'Missing authentication',
                    'severity': 'Critical',
                    'status_code': response.status_code
                })
            
            # Test 2: Null/Empty tokens
            headers = {'Authorization': 'Bearer '}
            response = requests.get(
                f"{self.base_url}{endpoint}",
                headers=headers
            )
            if response.status_code == 200:
                results.append({
                    'endpoint': endpoint,
                    'issue': 'Empty token accepted',
                    'severity': 'Critical'
                })
            
            # Test 3: Token manipulation (JWT algorithm confusion)
            if 'Authorization' in self.session.headers:
                token = self.session.headers['Authorization'].replace('Bearer ', '')
                if '.' in token:  # Likely JWT
                    # Try algorithm confusion attack (none/None)
                    parts = token.split('.')
                    if len(parts) == 3:
                        header = json.loads(base64.b64decode(parts[0] + '=='))
                        header['alg'] = 'none'
                        new_header = base64.b64encode(json.dumps(header).encode()).decode().rstrip('=')
                        malicious_token = f"{new_header}.{parts[1]}.{parts[2]}"
                        
                        resp = requests.get(
                            f"{self.base_url}{endpoint}",
                            headers={'Authorization': f'Bearer {malicious_token}'}
                        )
                        if resp.status_code == 200:
                            results.append({
                                'endpoint': endpoint,
                                'issue': 'JWT Algorithm Confusion (none)',
                                'severity': 'Critical'
                            })
        
        return results
    
    def test_authorization(self, user_endpoints, admin_endpoints):
        """
        Test horizontal and vertical authorization controls
        """
        results = []
        
        # Horizontal IDOR testing
        for endpoint in user_endpoints:
            # Try accessing other users' resources
            for id in [1, 2, 999]:
                response = self.session.get(f"{self.base_url}{endpoint}/{id}")
                if response.status_code == 200 and id != self.current_user_id():
                    results.append({
                        'type': 'IDOR',
                        'endpoint': f"{endpoint}/{id}",
                        'issue': 'Access to other user data',
                        'severity': 'High'
                    })
        
        # Vertical privilege escalation
        for endpoint in admin_endpoints:
            response = self.session.get(f"{self.base_url}{endpoint}")
            if response.status_code == 200:
                results.append({
                    'type': 'Vertical Escalation',
                    'endpoint': endpoint,
                    'issue': 'User can access admin functionality',
                    'severity': 'Critical'
                })
        
        return results
    
    def test_input_validation(self, endpoints_with_params):
        """
        Test for injection and validation vulnerabilities
        """
        payloads = {
            'sql_injection': ["' OR '1'='1", "1 AND 1=1", "1 AND 1=2"],
            'xss': ["<script>alert(1)</script>", "<img src=x onerror=alert(1)>"],
            'command_injection': ["; id", "| whoami", "$(cat /etc/passwd)"],
            'path_traversal': ["../../../etc/passwd", "..\\..\\..\\windows\\system32\\config\\sam"],
            'ssrf': ["http://169.254.169.254/latest/meta-data/", "file:///etc/passwd"]
        }
        
        results = []
        
        for endpoint, param in endpoints_with_params:
            for attack_type, tests in payloads.items():
                for payload in tests:
                    data = {param: payload}
                    response = self.session.post(f"{self.base_url}{endpoint}", json=data)
                    
                    # Check for indicators of vulnerability
                    indicators = self.check_vulnerability_indicators(response, attack_type)
                    if indicators:
                        results.append({
                            'endpoint': endpoint,
                            'param': param,
                            'attack_type': attack_type,
                            'payload': payload,
                            'indicators': indicators,
                            'severity': 'High'
                        })
        
        return results
    
    def check_vulnerability_indicators(self, response, attack_type):
        """
        Check response for signs of successful injection
        """
        indicators = []
        content = response.text.lower()
        
        if attack_type == 'sql_injection':
            sql_errors = ['sql syntax', 'mysql_fetch', 'ora-', 'postgresql', 'sqlite3']
            if any(err in content for err in sql_errors):
                indicators.append('SQL error message detected')
            if response.status_code == 500:
                indicators.append('Server error (possible SQL error)')
        
        elif attack_type == 'command_injection':
            unix_indicators = ['uid=', 'gid=', 'root:x:', 'bin/bash']
            windows_indicators = ['windows ip configuration', 'volume serial number']
            if any(ind in content for ind in unix_indicators + windows_indicators):
                indicators.append('Command output detected')
        
        elif attack_type == 'xss':
            if '<script>' in response.text and 'alert' in response.text:
                indicators.append('XSS payload reflected without encoding')
        
        elif attack_type == 'ssrf':
            if 'amazon' in content or 'ec2' in content or 'metadata' in content:
                indicators.append('Cloud metadata accessible')
        
        return indicators
    
    def test_mass_assignment(self, endpoint, valid_payload):
        """
        Test for mass assignment vulnerabilities (binding client data to model)
        """
        # Add sensitive fields that shouldn't be user-modifiable
        malicious_payload = valid_payload.copy()
        malicious_payload.update({
            'is_admin': True,
            'role': 'admin',
            'id': 1,
            'created_at': '2020-01-01',
            'password': 'hacked123',
            'credit': 999999
        })
        
        response = self.session.post(f"{self.base_url}{endpoint}", json=malicious_payload)
        
        # Check if sensitive fields were accepted
        get_response = self.session.get(f"{self.base_url}{endpoint}/profile")
        
        findings = []
        if 'is_admin' in get_response.text and '"is_admin": true' in get_response.text:
            findings.append({
                'issue': 'Mass Assignment - is_admin field accepted',
                'severity': 'Critical'
            })
        
        return findings
    
    def test_rate_limiting(self, endpoint, requests_count=100):
        """
        Test for API rate limiting
        """
        responses = []
        for i in range(requests_count):
            response = self.session.get(f"{self.base_url}{endpoint}")
            responses.append(response.status_code)
            
            if response.status_code in [429, 503]:
                return {
                    'rate_limited': True,
                    'threshold': i + 1,
                    'reset_header': response.headers.get('Retry-After'),
                    'status': 'Protected'
                }
        
        unique_codes = set(responses)
        if 200 in unique_codes and len(unique_codes) == 1:
            return {
                'rate_limited': False,
                'total_requests': requests_count,
                'status': 'VULNERABLE - No rate limiting detected',
                'severity': 'Medium'
            }
        
        return {'rate_limited': True, 'status': 'Protected'}
    
    def current_user_id(self):
        """Helper to get current user ID from token/session"""
        # Implementation depends on API structure
        return 1
```

### **40.3.2 GraphQL Security Testing**

```python
# GraphQL Specific Security Testing
class GraphQLTester:
    def __init__(self, endpoint):
        self.endpoint = endpoint
    
    def introspection_query(self):
        """
        Check if introspection is enabled (information disclosure)
        """
        query = """
        query IntrospectionQuery {
          __schema {
            queryType { name }
            mutationType { name }
            subscriptionType { name }
            types {
              ...FullType
            }
          }
        }
        fragment FullType on __Type {
          name
          kind
          fields {
            name
            type { name }
            args {
              name
              type { name }
            }
          }
        }
        """
        
        response = requests.post(
            self.endpoint,
            json={'query': query}
        )
        
        if response.status_code == 200 and '__schema' in response.text:
            return {
                'vulnerability': 'Introspection Enabled',
                'severity': 'Medium',
                'description': 'Schema can be dumped revealing all types and fields',
                'data': response.json()
            }
        return None
    
    def test_query_depth(self, max_depth=10):
        """
        Test for query depth limiting (DoS prevention)
        """
        # Build nested query
        nested_query = "user {"
        for _ in range(max_depth):
            nested_query += " friends {"
        nested_query += " name " + "}" * (max_depth + 1)
        
        query = f"{{ {nested_query} }}"
        
        response = requests.post(
            self.endpoint,
            json={'query': query}
        )
        
        if response.status_code == 200:
            return {
                'vulnerability': 'No Query Depth Limit',
                'severity': 'Medium',
                'description': f'Query with depth {max_depth} was accepted, potential DoS'
            }
        return None
    
    def test_batch_queries(self):
        """
        Test for query batching (can bypass rate limiting)
        """
        queries = [
            {'query': '{ user(id: 1) { name } }'},
            {'query': '{ user(id: 2) { name } }'},
            {'query': '{ user(id: 3) { name } }'}
        ]
        
        response = requests.post(
            self.endpoint,
            json=queries  # Send array instead of single object
        )
        
        if response.status_code == 200:
            data = response.json()
            if isinstance(data, list) and len(data) == len(queries):
                return {
                    'vulnerability': 'Query Batching Enabled',
                    'severity': 'Low',
                    'description': 'Multiple queries accepted in single request, can bypass rate limits'
                }
        return None
    
    def test_field_suggestions(self):
        """
        Check if field suggestions leak schema information
        """
        query = "{ usr { name } }"  # Typo: usr instead of user
        
        response = requests.post(
            self.endpoint,
            json={'query': query}
        )
        
        if 'did you mean' in response.text.lower() or 'user' in response.text:
            return {
                'vulnerability': 'Field Suggestions Enabled',
                'severity': 'Info',
                'description': 'Error messages reveal valid field names'
            }
        return None
```

---

## **40.4 Mobile Application Security Testing**

### **40.4.1 Mobile Testing Methodology**

```python
# Mobile App Security Testing Framework
class MobileSecurityTester:
    """
    Mobile App Security Testing (MAST) methodology
    Covers OWASP Mobile Top 10 and MASVS (Mobile Application Security Verification Standard)
    """
    
    def __init__(self, app_binary, platform='android'):
        self.app = app_binary
        self.platform = platform.lower()
        self.findings = []
    
    def static_analysis(self):
        """
        Static Application Security Testing (SAST) for mobile
        """
        results = {
            'hardcoded_secrets': self.find_hardcoded_secrets(),
            'insecure_storage': self.check_insecure_storage(),
            'code_obfuscation': self.check_obfuscation(),
            'debug_flags': self.check_debug_flags(),
            'backup_vulnerabilities': self.check_backup_issues()
        }
        return results
    
    def find_hardcoded_secrets(self):
        """
        Search for hardcoded API keys, passwords, tokens in binary
        """
        patterns = [
            r'api[_-]?key\s*[=:]\s*["\'][a-zA-Z0-9]{16,}["\']',
            r'password\s*[=:]\s*["\'][^"\']+["\']',
            r'aws_access_key_id\s*[=:]\s*["\']AKIA[0-9A-Z]{16}["\']',
            r'private[_-]?key',
            r'secret[_-]?key'
        ]
        
        findings = []
        # Implementation would use strings command or apktool/jadx output
        # Example with strings command:
        try:
            output = subprocess.check_output(['strings', self.app]).decode('utf-8', errors='ignore')
            for pattern in patterns:
                matches = re.findall(pattern, output, re.IGNORECASE)
                if matches:
                    findings.append({
                        'pattern': pattern,
                        'matches': matches[:5],  # Limit output
                        'severity': 'High'
                    })
        except Exception as e:
            findings.append({'error': str(e)})
        
        return findings
    
    def check_insecure_storage(self):
        """
        Check for insecure local data storage
        """
        issues = []
        
        # Check for SharedPreferences (Android) or UserDefaults (iOS)
        if self.platform == 'android':
            # Look for MODE_WORLD_READABLE or MODE_WORLD_WRITEABLE
            if 'MODE_WORLD_READABLE' in open(self.app, 'rb').read().decode('utf-8', errors='ignore'):
                issues.append('World-readable SharedPreferences detected')
            
            # Check for external storage usage
            if 'getExternalStorageDirectory' in str(subprocess.check_output(['unzip', '-l', self.app])):
                issues.append('External storage usage detected - potential data leakage')
        
        return issues
    
    def dynamic_analysis_setup(self):
        """
        Setup for dynamic testing (Frida/Objection setup)
        """
        setup_commands = {
            'android': [
                'adb devices',  # Verify device connection
                'adb root',     # Root access
                'frida-server &',  # Start Frida server
                'objection --gadget com.example.app explore'  # Start Objection
            ],
            'ios': [
                'ios-deploy --list',  # List devices
                'frida -U -f com.example.app',  # Spawn app with Frida
                'objection --gadget com.example.app explore'
            ]
        }
        
        return setup_commands.get(self.platform, [])
    
    def test_ssl_pinning_bypass(self):
        """
        SSL Pinning bypass techniques
        """
        techniques = {
            'android': [
                'Frida script: android-ssl-pinning-disable',
                'Objection: android sslpinning disable',
                'Xposed module: SSLUnpinning',
                'APK patching: Remove pinning logic from smali'
            ],
            'ios': [
                'Frida script: ios-ssl-pinning-disable',
                'Objection: ios sslpinning disable',
                'SSL Kill Switch 2 (Cydia tweak)',
                'Binary patching: nop out pinning checks'
            ]
        }
        
        return {
            'platform': self.platform,
            'bypass_techniques': techniques.get(self.platform),
            'test_script': self.generate_frida_ssl_script()
        }
    
    def generate_frida_ssl_script(self):
        """
        Generate Frida script for SSL pinning bypass
        """
        if self.platform == 'android':
            return """
            Java.perform(function() {
                var array_list = Java.use("java.util.ArrayList");
                var TrustManagerImpl = Java.use("com.android.org.conscrypt.TrustManagerImpl");
                
                TrustManagerImpl.checkTrustedRecursive.implementation = function() {
                    return array_list.$new();
                };
            });
            """
        else:
            return """
            var SSL pinning bypass for iOS
            """
    
    def generate_mobsf_report(self):
        """
        Generate report compatible with Mobile Security Framework
        """
        return {
            'file_name': self.app,
            'platform': self.platform,
            'security_score': self.calculate_score(),
            'findings': self.findings,
            'compliance': {
                'OWASP_MASVS': self.check_masvs_compliance(),
                'OWASP_Mobile_Top_10': self.check_mobile_top10()
            }
        }
    
    def calculate_score(self):
        """Calculate security score based on findings"""
        base_score = 100
        for finding in self.findings:
            if finding['severity'] == 'Critical':
                base_score -= 20
            elif finding['severity'] == 'High':
                base_score -= 10
            elif finding['severity'] == 'Medium':
                base_score -= 5
            else:
                base_score -= 1
        return max(0, base_score)
    
    def check_masvs_compliance(self):
        """Check against Mobile Application Security Verification Standard"""
        # MASVS Levels: L1 (Standard), L2 (Defense in Depth), L3 (Resilient)
        return {
            'L1_Storage': 'Pass' if not self.find_hardcoded_secrets() else 'Fail',
            'L1_Crypto': 'Pass',  # Would check crypto implementation
            'L1_Auth': 'Pass',    # Would check auth mechanisms
            'L1_Network': 'Pass' if self.test_ssl_pinning_bypass() else 'Fail'
        }
```

---

## **40.5 Cloud Security Testing**

### **40.5.1 AWS Security Assessment**

```python
# AWS Cloud Security Testing
import boto3
from botocore.exceptions import ClientError

class AWSSecurityTester:
    def __init__(self, profile_name='default'):
        self.session = boto3.Session(profile_name=profile_name)
        self.findings = []
    
    def s3_security_audit(self):
        """
        S3 Bucket Security Assessment
        """
        s3 = self.session.client('s3')
        findings = []
        
        try:
            buckets = s3.list_buckets()['Buckets']
            
            for bucket in buckets:
                bucket_name = bucket['Name']
                issues = []
                
                # Check public access settings
                try:
                    public_access = s3.get_public_access_block(Bucket=bucket_name)
                    config = public_access['PublicAccessBlockConfiguration']
                    if not all(config.values()):
                        issues.append('Public access block not fully enabled')
                except ClientError:
                    issues.append('No public access block configured')
                
                # Check bucket policy
                try:
                    policy = s3.get_bucket_policy(Bucket=bucket_name)
                    if '"Principal": "*"' in policy['Policy']:
                        issues.append('Bucket policy allows public access')
                except ClientError as e:
                    if 'NoSuchBucketPolicy' not in str(e):
                        raise
                
                # Check bucket ACL
                try:
                    acl = s3.get_bucket_acl(Bucket=bucket_name)
                    for grant in acl['Grants']:
                        if grant['Grantee'].get('URI') == 'http://acs.amazonaws.com/groups/global/AllUsers':
                            issues.append('Bucket ACL grants public access')
                except ClientError:
                    pass
                
                # Check encryption
                try:
                    encryption = s3.get_bucket_encryption(Bucket=bucket_name)
                except ClientError as e:
                    if 'ServerSideEncryptionConfigurationNotFoundError' in str(e):
                        issues.append('Bucket encryption not enabled')
                
                # Check versioning
                versioning = s3.get_bucket_versioning(Bucket=bucket_name)
                if versioning.get('Status') != 'Enabled':
                    issues.append('Versioning not enabled - ransomware risk')
                
                if issues:
                    findings.append({
                        'bucket': bucket_name,
                        'issues': issues,
                        'severity': 'High' if 'public' in str(issues).lower() else 'Medium'
                    })
        
        except ClientError as e:
            findings.append({'error': str(e)})
        
        return findings
    
    def iam_security_audit(self):
        """
        IAM Security Assessment
        """
        iam = self.session.client('iam')
        findings = []
        
        # Check for old passwords/access keys
        users = iam.list_users()['Users']
        for user in users:
            username = user['UserName']
            
            # Check password age
            if 'PasswordLastUsed' in user:
                last_used = user['PasswordLastUsed']
                age_days = (datetime.now(last_used.tzinfo) - last_used).days
                if age_days > 90:
                    findings.append({
                        'user': username,
                        'issue': f'Password not changed in {age_days} days',
                        'severity': 'Medium'
                    })
            
            # Check access keys
            keys = iam.list_access_keys(UserName=username)['AccessKeyMetadata']
            for key in keys:
                if key['Status'] == 'Active':
                    key_age = (datetime.now(key['CreateDate'].tzinfo) - key['CreateDate']).days
                    if key_age > 90:
                        findings.append({
                            'user': username,
                            'issue': f'Access key {key["AccessKeyId"][:5]}... is {key_age} days old',
                            'severity': 'Medium'
                        })
        
        # Check for MFA on root account
        try:
            summary = iam.get_account_summary()['SummaryMap']
            if summary.get('AccountMFAEnabled') == 0:
                findings.append({
                    'resource': 'Root Account',
                    'issue': 'MFA not enabled on root account',
                    'severity': 'Critical'
                })
        except ClientError:
            pass
        
        return findings
    
    def ec2_security_audit(self):
        """
        EC2 Instance Security Assessment
        """
        ec2 = self.session.client('ec2')
        findings = []
        
        # Check security groups for open ports
        security_groups = ec2.describe_security_groups()['SecurityGroups']
        for sg in security_groups:
            for rule in sg['IpPermissions']:
                # Check for 0.0.0.0/0
                for ip_range in rule.get('IpRanges', []):
                    if ip_range.get('CidrIp') == '0.0.0.0/0':
                        port_range = f"{rule.get('FromPort', 'All')}-{rule.get('ToPort', 'All')}"
                        protocol = rule.get('IpProtocol')
                        
                        # Critical if SSH or RDP open
                        severity = 'Critical' if port_range in ['22-22', '3389-3389'] else 'High'
                        
                        findings.append({
                            'security_group': sg['GroupId'],
                            'issue': f'Open to world: {protocol} {port_range}',
                            'severity': severity
                        })
        
        # Check for unencrypted volumes
        volumes = ec2.describe_volumes()['Volumes']
        for vol in volumes:
            if not vol.get('Encrypted'):
                findings.append({
                    'volume': vol['VolumeId'],
                    'issue': 'EBS volume not encrypted',
                    'severity': 'Medium'
                })
        
        return findings
    
    def generate_cloud_security_report(self):
        """
        Generate comprehensive cloud security report
        """
        return {
            'scan_date': datetime.now().isoformat(),
            'aws_account': self.session.client('sts').get_caller_identity()['Account'],
            'findings': {
                's3': self.s3_security_audit(),
                'iam': self.iam_security_audit(),
                'ec2': self.ec2_security_audit()
            },
            'compliance': {
                'cis_aws_foundations': self.check_cis_compliance(),
                'pci_dss': self.check_pci_compliance()
            }
        }
```

### **40.5.2 Container Security Testing**

```python
# Docker/Kubernetes Security Testing
class ContainerSecurityTester:
    def __init__(self, image_name):
        self.image = image_name
    
    def image_vulnerability_scan(self):
        """
        Scan container image for vulnerabilities using Trivy
        """
        cmd = ['trivy', 'image', '--format', 'json', self.image]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True)
            data = json.loads(result.stdout)
            
            vulnerabilities = []
            for result in data.get('Results', []):
                for vuln in result.get('Vulnerabilities', []):
                    vulnerabilities.append({
                        'id': vuln.get('VulnerabilityID'),
                        'package': vuln.get('PkgName'),
                        'severity': vuln.get('Severity'),
                        'title': vuln.get('Title'),
                        'fixed_version': vuln.get('FixedVersion')
                    })
            
            return vulnerabilities
        except Exception as e:
            return [{'error': str(e)}]
    
    def dockerfile_best_practices(self, dockerfile_path):
        """
        Check Dockerfile for security best practices
        """
        with open(dockerfile_path, 'r') as f:
            content = f.read()
        
        findings = []
        
        # Check for latest tag
        if ':latest' in content or 'FROM ' in content and ':' not in content:
            findings.append({
                'issue': 'Using "latest" tag',
                'recommendation': 'Pin to specific version',
                'severity': 'Medium'
            })
        
        # Check for running as root
        if 'USER ' not in content:
            findings.append({
                'issue': 'No USER instruction - running as root',
                'recommendation': 'Add non-root user',
                'severity': 'High'
            })
        
        # Check for secrets in ENV
        if re.search(r'ENV.*(password|secret|key|token)', content, re.IGNORECASE):
            findings.append({
                'issue': 'Potential hardcoded secrets in ENV',
                'recommendation': 'Use build args or runtime secrets',
                'severity': 'Critical'
            })
        
        # Check for ADD vs COPY
        if 'ADD ' in content and 'http' in content:
            findings.append({
                'issue': 'ADD with remote URL',
                'recommendation': 'Use curl/wget with checksum verification',
                'severity': 'Medium'
            })
        
        return findings
    
    def kubernetes_manifest_check(self, manifest_path):
        """
        Check Kubernetes manifests for security issues
        """
        import yaml
        
        with open(manifest_path, 'r') as f:
            manifest = yaml.safe_load(f)
        
        findings = []
        
        spec = manifest.get('spec', {})
        containers = spec.get('containers', [])
        
        for container in containers:
            # Check for privileged mode
            security_context = container.get('securityContext', {})
            if security_context.get('privileged'):
                findings.append({
                    'container': container.get('name'),
                    'issue': 'Running in privileged mode',
                    'severity': 'Critical'
                })
            
            # Check for read-only root filesystem
            if not security_context.get('readOnlyRootFilesystem'):
                findings.append({
                    'container': container.get('name'),
                    'issue': 'Root filesystem not read-only',
                    'severity': 'Medium'
                })
            
            # Check resource limits
            resources = container.get('resources', {})
            if not resources.get('limits'):
                findings.append({
                    'container': container.get('name'),
                    'issue': 'No resource limits set',
                    'severity': 'Low'
                })
        
        # Check for secrets in environment variables
        env = container.get('env', [])
        for var in env:
            if 'value' in var and any(keyword in var.get('name', '').lower() 
                                     for keyword in ['password', 'secret', 'key', 'token']):
                findings.append({
                    'issue': f'Secret {var["name"]} exposed in environment variable',
                    'recommendation': 'Use Kubernetes Secrets',
                    'severity': 'High'
                })
        
        return findings
```

---

## **40.6 Bug Bounty Methodology**

### **40.6.1 Reconnaissance and Asset Discovery**

```python
# Bug Bounty Automation Framework
class BugBountyRecon:
    def __init__(self, target_domain):
        self.domain = target_domain
        self.subdomains = set()
        self.endpoints = set()
        self.tech_stack = {}
    
    def comprehensive_recon(self):
        """
        Comprehensive reconnaissance for bug bounty programs
        """
        results = {
            'subdomains': self.enumerate_subdomains(),
            'endpoints': self.discover_endpoints(),
            'js_analysis': self.analyze_javascript(),
            'github_recon': self.github_reconnaissance(),
            'cloud_assets': self.find_cloud_assets()
        }
        return results
    
    def enumerate_subdomains(self):
        """
        Multi-source subdomain enumeration
        """
        tools = {
            'subfinder': f'subfinder -d {self.domain} -all -o subfinder.txt',
            'amass': f'amass enum -passive -d {self.domain} -o amass.txt',
            'assetfinder': f'assetfinder --subs-only {self.domain} > assetfinder.txt',
            'chaos': f'chaos -d {self.domain} -o chaos.txt'
        }
        
        # Run tools and aggregate results
        for tool_name, command in tools.items():
            try:
                subprocess.run(command, shell=True, timeout=300)
                with open(f'{tool_name}.txt', 'r') as f:
                    self.subdomains.update(line.strip() for line in f)
            except Exception as e:
                print(f"{tool_name} failed: {e}")
        
        # Verify subdomains are alive
        live_hosts = []
        for sub in self.subdomains:
            try:
                response = requests.get(f'http://{sub}', timeout=5)
                live_hosts.append({
                    'subdomain': sub,
                    'status': response.status_code,
                    'title': self.get_page_title(response.text)
                })
            except:
                pass
        
        return live_hosts
    
    def analyze_javascript(self):
        """
        Extract endpoints and secrets from JavaScript files
        """
        js_endpoints = set()
        secrets = []
        
        # Find JS files
        response = requests.get(f'https://{self.domain}')
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script', src=True)
        
        for script in scripts:
            if script['src'].endswith('.js'):
                js_url = script['src'] if script['src'].startswith('http') else f"{self.domain}{script['src']}"
                
                try:
                    js_content = requests.get(js_url).text
                    
                    # Extract API endpoints
                    api_patterns = re.findall(r'["\'](\/api\/[a-zA-Z0-9\/_-]+)["\']', js_content)
                    js_endpoints.update(api_patterns)
                    
                    # Extract secrets
                    secret_patterns = [
                        (r'api[_-]?key["\']?\s*:\s*["\']([a-zA-Z0-9]{16,})["\']', 'API Key'),
                        (r'secret["\']?\s*:\s*["\']([a-zA-Z0-9]{16,})["\']', 'Secret'),
                        (r'auth[_-]?token["\']?\s*:\s*["\']([a-zA-Z0-9-_\.]+)["\']', 'Auth Token')
                    ]
                    
                    for pattern, secret_type in secret_patterns:
                        matches = re.findall(pattern, js_content, re.IGNORECASE)
                        for match in matches:
                            secrets.append({
                                'type': secret_type,
                                'value': match[:10] + '...',
                                'file': js_url
                            })
                    
                    # Extract S3 buckets, GCP buckets
                    buckets = re.findall(r'([a-z0-9-]+\.s3\.amazonaws\.com)', js_content)
                    
                except Exception as e:
                    continue
        
        return {
            'endpoints': list(js_endpoints),
            'secrets': secrets,
            's3_buckets': list(set(buckets)) if 'buckets' in locals() else []
        }
    
    def github_reconnaissance(self):
        """
        Search GitHub for leaked credentials and code
        """
        # Using GitHub API or tools like GitHound
        queries = [
            f'"{self.domain}" password',
            f'"{self.domain}" api_key',
            f'"{self.domain}" token',
            f'"{self.domain}" database_password',
            f'org:{self.domain.split(".")[0]} secret'
        ]
        
        findings = []
        # Implementation would use GitHub API with proper authentication
        # to avoid rate limits
        
        return findings
    
    def get_page_title(self, html):
        """Helper to extract page title"""
        try:
            soup = BeautifulSoup(html, 'html.parser')
            return soup.title.string if soup.title else 'No title'
        except:
            return 'Error parsing'
```

### **40.6.2 Vulnerability Triage and Reporting**

```python
class BugBountyReporter:
    """
    Generate professional bug bounty reports
    """
    
    def __init__(self, vulnerability_data):
        self.vuln = vulnerability_data
    
    def generate_report(self):
        """
        Generate markdown report for bug bounty platforms
        """
        template = """
# {title}

## Summary
{summary}

## Severity
{severity} ({cvss_score})

## Steps to Reproduce
{steps}

## Impact
{impact}

## Proof of Concept
{poc}

## Remediation
{remediation}

## References
{references}
        """
        
        return template.format(
            title=self.vuln['title'],
            summary=self.vuln['summary'],
            severity=self.vuln['severity'],
            cvss_score=self.vuln.get('cvss', 'N/A'),
            steps=self.format_steps(self.vuln['steps']),
            impact=self.vuln['impact'],
            poc=self.vuln.get('poc', 'See attached screenshots'),
            remediation=self.vuln['remediation'],
            references=self.vuln.get('references', 'OWASP guidelines')
        )
    
    def format_steps(self, steps):
        """Format reproduction steps as numbered list"""
        return '\n'.join(f'{i+1}. {step}' for i, step in enumerate(steps))
    
    def calculate_bounty_range(self):
        """
        Suggest bounty range based on severity and impact
        """
        severity_multipliers = {
            'Critical': (5000, 10000),
            'High': (1000, 5000),
            'Medium': (300, 1000),
            'Low': (100, 300),
            'Informational': (0, 0)
        }
        
        base = severity_multipliers.get(self.vuln['severity'], (0, 0))
        
        # Adjust for business impact
        if 'customer data' in self.vuln['impact'].lower():
            base = (base[0] * 1.5, base[1] * 1.5)
        
        return {
            'suggested_range': f"${int(base[0])} - ${int(base[1])}",
            'factors': ['Severity', 'Business impact', 'Exploitation difficulty']
        }
```

---

## **Chapter Summary**

In this comprehensive chapter on **Security Testing Techniques**, we covered the methodologies and frameworks that guide professional security assessments:

**Key Methodologies:**

1. **OWASP Testing Guide (40.1):** The industry-standard framework covering 12 categories from Information Gathering (DNS enumeration, fingerprinting, endpoint discovery) through Configuration Management (security headers, HTTP methods), Identity Management, Authentication (session management, password policies), Authorization (IDOR testing), Input Validation (SQLi, XSS, Command Injection), and Business Logic testing.

2. **PTES - Penetration Testing Execution Standard (40.2):** A comprehensive 7-phase approach covering Pre-engagement (scoping, ROE, legal), Intelligence Gathering (OSINT, footprinting), Threat Modeling (STRIDE methodology), Vulnerability Analysis, Exploitation, Post-exploitation (pivoting, data analysis), and Reporting. We implemented Python tools for threat modeling using STRIDE and risk calculation.

3. **API Security Testing (40.3):** Specialized techniques for REST APIs (authentication bypass, authorization testing, mass assignment, rate limiting) and GraphQL (introspection queries, depth limiting, batch query attacks). We built comprehensive testers for JWT vulnerabilities, IDOR detection, and injection attacks specific to API endpoints.

4. **Mobile Security Testing (40.4):** MAST (Mobile Application Security Testing) methodology covering OWASP Mobile Top 10 and MASVS. We covered static analysis (hardcoded secrets, insecure storage), dynamic analysis setup (Frida/Objection), SSL pinning bypass techniques, and platform-specific testing for Android and iOS.

5. **Cloud Security Testing (40.5):** AWS security assessment tools for S3 buckets (public access, encryption), IAM (MFA, key rotation), EC2 (security groups, encryption), and Container Security (Dockerfile best practices, Kubernetes manifest security, vulnerability scanning with Trivy).

6. **Bug Bounty Methodology (40.6):** Reconnaissance automation (subdomain enumeration, JavaScript analysis, GitHub reconnaissance), vulnerability triage, and professional report generation with severity classification and bounty range estimation.

**Industry Standards Compliance:**
- **NIST SP 800-115:** Technical Guide to Information Security Testing
- **OWASP MASVS:** Mobile Application Security Verification Standard
- **CIS Benchmarks:** Cloud security configuration standards
- **PCI-DSS:** Payment card industry security requirements

---

## **📖 Next Chapter: Chapter 41 - Test-Driven Development (TDD)**

Now that you have mastered security testing methodologies, Chapter 41 will transition to **development methodologies that prevent vulnerabilities from being introduced in the first place**.

In **Chapter 41**, you will learn:

- **Test-Driven Development (TDD):** The Red-Green-Refactor cycle, writing tests before code, and how TDD naturally prevents security vulnerabilities by forcing developers to consider edge cases and invalid inputs.

- **Unit Testing Fundamentals:** Writing effective unit tests, test isolation, mocking and stubbing, and achieving meaningful code coverage without chasing vanity metrics.

- **TDD Security:** How to write security-focused unit tests (input validation tests, authentication logic tests, authorization boundary tests) that catch vulnerabilities at the earliest possible stage.

- **Behavior-Driven Development (BDD):** Extending TDD with business-readable specifications using Gherkin syntax, and how BDD scenarios can capture security requirements in business terms.

- **Secure Coding Practices:** Input validation, output encoding, secure authentication patterns, and defensive programming techniques integrated into the TDD workflow.

- **Refactoring for Security:** How to safely refactor code to improve security posture without breaking functionality, using tests as safety nets.

**Chapter 41 will teach you how to build security into the development process from day one, ensuring that quality and security are inherent in your codebase rather than retrofitted through testing alone.**

**Continue to Chapter 41 to learn how to write code that is secure by design through Test-Driven Development!**